In [1]:
# !pip install colossalai

## Initialization

In [1]:
import os
from pathlib import Path
import math
import colossalai
import torch
import torch.nn as nn
import torch.nn.functional as F
from colossalai.core import global_context as gpc
from colossalai.logging import get_dist_logger
from colossalai.nn import CosineAnnealingLR
from colossalai.nn.metric import Accuracy
from colossalai.trainer import Trainer, hooks
from colossalai.utils import MultiTimer, get_dataloader
from torchvision import transforms
from torchvision.datasets import MNIST
from tqdm import tqdm

class LeNet5(nn.Module):
    def __init__(self, n_classes):
        super(LeNet5, self).__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits


config = {'BATCH_SIZE':128,'NUM_EPOCHS':10}

colossalai.launch(config=config,rank=0,world_size=1,host='127.0.0.1',port=1235)

logger = get_dist_logger()

# build 

model = LeNet5(n_classes=10)

# build dataloaders
train_dataset = MNIST(
    root=Path('./tmp/'),
    download=True,
    transform = transforms.Compose([transforms.Resize((32, 32)),
                              transforms.ToTensor()])
)

test_dataset = MNIST(
    root=Path('./tmp/'),
    train=False,
    transform = transforms.Compose([transforms.Resize((32, 32)),
                              transforms.ToTensor()])
)

train_dataloader = get_dataloader(dataset=train_dataset,
                                  shuffle=True,
                                  batch_size=gpc.config.BATCH_SIZE,
                                  num_workers=1,
                                  pin_memory=True,
                                  )

test_dataloader = get_dataloader(dataset=test_dataset,
                                  add_sampler=False,
                                  batch_size=gpc.config.BATCH_SIZE,
                                  num_workers=1,
                                  pin_memory=True,
                                  )

# build criterion
criterion = torch.nn.CrossEntropyLoss()

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

#exponentially increase learning rate from low to high
def lrs(batch):
    low = math.log2(1e-5)
    high = math.log2(10)
    return 2**(low+(high-low)*batch/len(train_dataloader)/gpc.config.NUM_EPOCHS)

# lr_scheduler
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lrs)

engine, train_dataloader, test_dataloader, _ = colossalai.initialize(model,
                                                                      optimizer,
                                                                      criterion,
                                                                      train_dataloader,
                                                                      test_dataloader,
                                                                      )
# build a timer to measure time
timer = MultiTimer()

# create a trainer object
trainer = Trainer(
    engine=engine,
    timer=timer,
    logger=logger
)

# define the hooks to attach to the trainer
hook_list = [
    hooks.LossHook(),
    hooks.LRSchedulerHook(lr_scheduler=lr_scheduler, by_epoch=False),
    # hooks.AccuracyHook(accuracy_func=Accuracy()),
    hooks.LogMetricByEpochHook(logger),
    hooks.LogMemoryByEpochHook(logger),
    hooks.LogTimingByEpochHook(timer, logger),

    # you can uncomment these lines if you wish to use them
    hooks.TensorboardHook(log_dir='./tb_logs', ranks=[0]),
    # hooks.SaveCheckpointHook(checkpoint_dir='./ckpt')
]

# start training
trainer.fit(
    train_dataloader=train_dataloader,
    epochs=gpc.config.NUM_EPOCHS,
    test_dataloader=test_dataloader,
    test_interval=1,
    hooks=hook_list,
    display_progress=True
)

Colossalai should be built with cuda extension to use the FP16 optimizer
If you want to activate cuda mode for MoE, please install with cuda_ext!


[04/14/22 16:43:13] INFO     colossalai - root - INFO: Added key: store_based_barrier_key:1  
                             to store for rank: 0

                    INFO     colossalai - root - INFO: Added key: store_based_barrier_key:2  
                             to store for rank: 0

                    INFO     colossalai - root - INFO: Added key: store_based_barrier_key:3  
                             to store for rank: 0

                    INFO     colossalai - root - INFO: Added key: store_based_barrier_key:4  
                             to store for rank: 0

...

                    INFO     colossalai - colossalai - INFO: [Epoch 4 / Test]: Loss = 0.89499

                    INFO     colossalai - colossalai - INFO: /home/lishen/Programme/anaconda3
                             /envs/torch18/lib/python3.7/site-packages/colossalai/trainer/hoo
                             ks/_log_hook.py:267 after_test_epoch

                    INFO     colossalai - colossalai - INFO: [Epoch 4 / Test]: Test-epoch:   
                             last = 1.5914 s, mean = 1.5914 s | Test-step: last = 0.0028422  
                             s, mean = 0.019907 s

                    INFO     colossalai - colossalai - INFO: /home/lishen/Programme/anaconda3
                             /envs/torch18/lib/python3.7/site-packages/colossalai/utils/memor
                             y_utils/memory_monitor.py:63 report_memory_usage

                    INFO     colossalai - colossalai - INFO: [Epoch 4 / Test]: GPU: allocated
                             0.72 MB, max allocated 5.82 MB, cached: 52.0 MB, max cached:    
                             52.0 MB

In [ ]:
%matplotlib inline
%load_ext tensorboard
%tensorboard --logdir tb_logs

## No Scheduling

In [2]:
LR = 0.1261
NUM_EPOCHS = 30
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)

# define the hooks to attach to the trainer
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda _: 1)

engine, train_dataloader, test_dataloader, _ = colossalai.initialize(
    model, optimizer, criterion, train_dataloader, test_dataloader,
)

# build a timer to measure time
timer = MultiTimer()

# create a trainer object
trainer = Trainer(
    engine=engine,
    timer=timer,
    logger=logger
)


hook_list = [
    hooks.LossHook(),
    hooks.LRSchedulerHook(lr_scheduler=lr_scheduler, by_epoch=False),
    # hooks.AccuracyHook(accuracy_func=Accuracy()),
    hooks.LogMetricByEpochHook(logger),
    hooks.LogMemoryByEpochHook(logger),
    hooks.LogTimingByEpochHook(timer, logger),

    # you can uncomment these lines if you wish to use them
    hooks.TensorboardHook(log_dir='./tb_logs', ranks=[0]),
    # hooks.SaveCheckpointHook(checkpoint_dir='./ckpt')
]

# start training
trainer.fit(
    train_dataloader=train_dataloader,
    epochs=NUM_EPOCHS,
    test_dataloader=test_dataloader,
    test_interval=1,
    hooks=hook_list,
    display_progress=True
)

[04/14/22 16:44:40] INFO     colossalai - colossalai - INFO: /home/lishen/Programme/anaconda3
                             /envs/torch18/lib/python3.7/site-packages/colossalai/initialize.
                             py:244 initialize

                    INFO     colossalai - colossalai - INFO:                                 
                             ========== Your Config ========                                 
                             {'BATCH_SIZE': 128, 'NUM_EPOCHS': 5}                            
                             ================================                                
                    

                    INFO     colossalai - colossalai - INFO: /home/lishen/Programme/anaconda3
                             /envs/torch18/lib/python3.7/site-packages/colossalai/initialize.
                             py:252 initialize

                    INFO     colossalai - colossalai - INFO: cuDNN benchmark = True,         
                             deterministic = False

                    WARNING  colossalai - colossalai - WARNING: /home/lishen/Programme/anacon
                             da3/envs/torch18/lib/python3.7/site-packages/colossalai/initiali
                             ze.py:281 initialize

...

                    INFO     colossalai - colossalai - INFO: [Epoch 29 / Test]: Loss =       
                             0.037576

                    INFO     colossalai - colossalai - INFO: /home/lishen/Programme/anaconda3
                             /envs/torch18/lib/python3.7/site-packages/colossalai/trainer/hoo
                             ks/_log_hook.py:267 after_test_epoch

                    INFO     colossalai - colossalai - INFO: [Epoch 29 / Test]: Test-epoch:  
                             last = 1.5647 s, mean = 1.5647 s | Test-step: last = 0.0026205  
                             s, mean = 0.01955 s

                    INFO     colossalai - colossalai - INFO: /home/lishen/Programme/anaconda3
                             /envs/torch18/lib/python3.7/site-packages/colossalai/utils/memor
                             y_utils/memory_monitor.py:63 report_memory_usage

                    INFO     colossalai - colossalai - INFO: [Epoch 29 / Test]: GPU:         
                             allocated 0.72 MB, max allocated 5.82 MB, cached: 52.0 MB, max  
                             cached: 52.0 MB

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs

## OneCycle Scheduler

In [3]:
LR = 0.1261
NUM_EPOCHS = 30
config = {'BATCH_SIZE':128,'NUM_EPOCHS':NUM_EPOCHS}

# lr_scheduler
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4)
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=LR, steps_per_epoch=len(train_dataloader), epochs=NUM_EPOCHS)


engine, train_dataloader, test_dataloader, _ = colossalai.initialize(
    model, optimizer, criterion, train_dataloader, test_dataloader,
)

# build a timer to measure time
timer = MultiTimer()

# create a trainer object
trainer = Trainer(
    engine=engine,
    timer=timer,
    logger=logger
)

# define the hooks to attach to the trainer
hook_list = [
    hooks.LossHook(),
    hooks.LRSchedulerHook(lr_scheduler=lr_scheduler, by_epoch=False),
    # hooks.AccuracyHook(accuracy_func=Accuracy()),
    hooks.LogMetricByEpochHook(logger),
    hooks.LogMemoryByEpochHook(logger),
    hooks.LogTimingByEpochHook(timer, logger),

    # you can uncomment these lines if you wish to use them
    hooks.TensorboardHook(log_dir='./tb_logs', ranks=[0]),
    # hooks.SaveCheckpointHook(checkpoint_dir='./ckpt')
]

# start training
trainer.fit(
    train_dataloader=train_dataloader,
    epochs=NUM_EPOCHS,
    test_dataloader=test_dataloader,
    test_interval=1,
    hooks=hook_list,
    display_progress=True
)



                    INFO     colossalai - colossalai - INFO: /home/lishen/Programme/anaconda3
                             /envs/torch18/lib/python3.7/site-packages/colossalai/initialize.
                             py:244 initialize

                    INFO     colossalai - colossalai - INFO:                                 
                             ========== Your Config ========                                 
                             {'BATCH_SIZE': 128, 'NUM_EPOCHS': 5}                            
                             ================================                                
                    

                    INFO     colossalai - colossalai - INFO: /home/lishen/Programme/anaconda3
                             /envs/torch18/lib/python3.7/site-packages/colossalai/initialize.
                             py:252 initialize

                    INFO     colossalai - colossalai - INFO: cuDNN benchmark = True,         
                             deterministic = False

                    WARNING  colossalai - colossalai - WARNING: /home/lishen/Programme/anacon
                             da3/envs/torch18/lib/python3.7/site-packages/colossalai/initiali
                             ze.py:281 initialize

...

                    INFO     colossalai - colossalai - INFO: [Epoch 29 / Test]: Loss =       
                             0.022196

                    INFO     colossalai - colossalai - INFO: /home/lishen/Programme/anaconda3
                             /envs/torch18/lib/python3.7/site-packages/colossalai/trainer/hoo
                             ks/_log_hook.py:267 after_test_epoch

                    INFO     colossalai - colossalai - INFO: [Epoch 29 / Test]: Test-epoch:  
                             last = 1.5269 s, mean = 1.5269 s | Test-step: last = 0.0027392  
                             s, mean = 0.019093 s

                    INFO     colossalai - colossalai - INFO: /home/lishen/Programme/anaconda3
                             /envs/torch18/lib/python3.7/site-packages/colossalai/utils/memor
                             y_utils/memory_monitor.py:63 report_memory_usage

                    INFO     colossalai - colossalai - INFO: [Epoch 29 / Test]: GPU:         
                             allocated 16.68 MB, max allocated 21.78 MB, cached: 58.0 MB, max
                             cached: 58.0 MB

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs